In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm.auto import tqdm
import seaborn as sns
sns.set()
from sklearn.cluster import *
from sklearn import preprocessing
from minisom import MiniSom 

In [2]:
som = pd.read_csv('som.csv')
res = pd.read_csv('res.csv')

In [4]:
features = res[['SW','phie','GR','ILD','RHOB']]
data=preprocessing.scale(features.values)
feature_names = features.columns

In [265]:
def som_to_clusterable(features):
    
    scaled_data=preprocessing.scale(features.values)
    feature_names = features.columns
    
    n=int(np.sqrt(5*np.sqrt(len(features))))
    size = len(feature_names)
    
    _som = MiniSom(n, n, size, sigma=0.3, learning_rate=0.5) # initialization of nxn SOM
    _som.pca_weights_init(scaled_data)
    _som.train_random(scaled_data, 100)
    W = _som.get_weights()
    
    somdf=[]
    for i,feat in enumerate(features):
        tempdf=pd.DataFrame(W.T[i])
        tempdf['feature'] = feat
        somdf.append(tempdf)
    som = pd.concat(somdf)
    
    w=[]
    for i,feat in enumerate(features):
        w.append(np.array(som.loc[som['feature'] == feat].iloc[:,1:-1]))
    w = np.array(w)

    dx = w.shape[1]
    dy = w.shape[2]
    w_table=[]
    for i,feat in enumerate(features):
        for x in range(dx):
            for y in range(dy):
                temp = w[i][x,y]
                line = [x,y,temp,feat]
                w_table.append(line)
            
    df = pd.DataFrame(w_table)
    df.columns = ['x','y','value','feat']
    wt=[]
    for i,feat in enumerate(features):
        temp = df.loc[df['feat'] == feat].iloc[:,-2:-1].values
        wt.append(temp)
    wt=np.array(wt)
    wt = wt.reshape(wt.shape[0],wt.shape[1])
    wt = pd.DataFrame(wt.T)
    wt.columns = feature_names
    xt = df['x'].iloc[:(wt.shape[0])]
    yt = df['y'].iloc[:(wt.shape[0])]
    wt['x'] = pd.Series(xt)
    wt['y'] = pd.Series(yt)

    clusterable=wt
    return clusterable

In [274]:
def som_backwards_cluster_assign(features,clustered_som):
        n=int(np.sqrt(5*np.sqrt(len(features))))
        size = len(feature_names)
        
        a = np.zeros((n,n,size))
        return size

In [266]:
maap = som_to_clusterable(features)

kmeans = KMeans(5,init='k-means++')
maap['cluster'] = kmeans.fit_predict(maap.iloc[:,:-3])

In [268]:
#plt.scatter(maap['x'],maap['y'],c=maap['cluster'],marker='.',s=30,cmap='viridis')

In [275]:
som_backwards_cluster_assign(features,maap)

5